# Author

Martin Wegmann (martin.wegmann@unibe.ch)

The goal of this notebook is to pre-process the data for the machine learning prediction task

# Loading libraries

In [ ]:
import xarray as xr
import iris
import glob
import iris.plot as iplt
import pandas as pd
import cdo
CDO = cdo.Cdo()
import netCDF4
import os
import datetime
import shutil  
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
#from mpl_toolkits.basemap import Basemap

# Land-Sea SUPERMASK

The land-sea mask was created using the land-sea definition of multiple SST data sets regridded to a 4° resolution, such as 20CRv3, MPI-GE and ERA5. On top of that, we labelled grid boxes as "land" when the maximum over all time steps of the normalised SSTs in that data set is larger than 5 or less than -5 , assuming that this grid box is highly volatile and might cause some problems as input to a prediction. We then multiply all those land-sea masks of those different datasets to come up with just one land-sea mask. 

This was done seperatly for May and November SSTs.

The netcdf files for May and November should be in the GIT-Hub folder

# MPI-GE preprocessing

## HIST T2M

In [ ]:

mpige_dir="/Volumes/SPARK/mpi_ge/"
mpige_dir_t2m_hist="/Volumes/SPARK/mpi_ge/hist/t2m/atmos/"
#data_name=os.path.join(data_dir,"test.nc")
#os.chdir(path)

In [ ]:
member_index=[f"{i:03}" for i in range(1,101)]

In [ ]:
# lets see what data we have
arr = os.listdir(mpige_dir_t2m_hist)
input_paths=[]
for file in os.listdir(mpige_dir_t2m_hist):
    if file.endswith(".nc"):
        print(os.path.join(mpige_dir_t2m_hist, file))
        mpige_t2m_file=os.path.join(mpige_dir_t2m_hist, file)
        input_paths.append(mpige_t2m_file)

In [ ]:
jan_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_hist, "tas_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_jan.nc')
    jan_paths.append(output)
    

In [ ]:
jun_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_hist, "tas_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_jun.nc')
    jun_paths.append(output)

In [ ]:
jul_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_hist, "tas_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_jul.nc')
    jul_paths.append(output)

In [ ]:
aug_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_hist, "tas_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_aug.nc')
    aug_paths.append(output)

In [ ]:
feb_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_hist, "tas_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_feb.nc')
    feb_paths.append(output)

In [ ]:
dec_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_hist, "tas_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_dec.nc')
    dec_paths.append(output)


In [ ]:
# calculate dec means and cut them to the region we will use for the dec NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("12", input=" -selyear,1850/2004 " +input_paths[i], output=dec_paths[i])

CDO.cat(input=dec_paths[:member_index_length], output=mpige_dir_t2m_hist+"tas_Amon_traintest_dec.nc")

CDO.remapnn("r96x48",input=mpige_dir_t2m_hist+"tas_Amon_traintest_dec.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc")
CDO.timstd(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_timstd.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_anomnorm.nc")


In [ ]:
# calculate jan means and cut them to the region we will use for the jan NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("1", input=" -selyear,1851/2005 " +input_paths[i], output=jan_paths[i])

CDO.cat(input=jan_paths[:member_index_length], output=mpige_dir_t2m_hist+"tas_Amon_traintest_jan.nc")

CDO.remapnn("r96x48",input=mpige_dir_t2m_hist+"tas_Amon_traintest_jan.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc")
CDO.timstd(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_timstd.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_anomnorm.nc")



In [ ]:
# calculate feb means and cut them to the region we will use for the feb NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("2", input=" -selyear,1851/2005 " +input_paths[i], output=feb_paths[i])

CDO.cat(input=feb_paths[:member_index_length], output=mpige_dir_t2m_hist+"tas_Amon_traintest_feb.nc")

CDO.remapnn("r96x48",input=mpige_dir_t2m_hist+"tas_Amon_traintest_feb.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc")
CDO.timstd(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_timstd.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_anomnorm.nc")



In [ ]:
# calculate jun means and cut them to the region we will use for the jun NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("6", input=" -selyear,1850/2005 " +input_paths[i], output=jun_paths[i])

CDO.cat(input=jun_paths[:member_index_length], output=mpige_dir_t2m_hist+"tas_Amon_traintest_jun.nc")

CDO.remapnn("r96x48",input=mpige_dir_t2m_hist+"tas_Amon_traintest_jun.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc")
CDO.timstd(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_timstd.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_anomnorm.nc")




In [ ]:
# calculate jul means and cut them to the region we will use for the jul NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("7", input=" -selyear,1850/2005 " +input_paths[i], output=jul_paths[i])

CDO.cat(input=jul_paths[:member_index_length], output=mpige_dir_t2m_hist+"tas_Amon_traintest_jul.nc")

CDO.remapnn("r96x48",input=mpige_dir_t2m_hist+"tas_Amon_traintest_jul.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc")
CDO.timstd(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_timstd.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_anomnorm.nc")


In [ ]:
# calculate aug means and cut them to the region we will use for the aug NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("8", input=" -selyear,1850/2005 " +input_paths[i], output=aug_paths[i])

CDO.cat(input=aug_paths[:member_index_length], output=mpige_dir_t2m_hist+"tas_Amon_traintest_aug.nc")

CDO.remapnn("r96x48",input=mpige_dir_t2m_hist+"tas_Amon_traintest_aug.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc")
CDO.timstd(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_timstd.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_anomnorm.nc")


In [ ]:
JJA_paths=[]
member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_hist, "tas_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_JJA.nc')
    JJA_paths.append(output)for i in range(member_index_length):
    CDO.timselmean("3", input="-selmon,6,7,8 " + input_paths[i], output=JJA_paths[i])
        
CDO.cat(input=JJA_paths[:member_index_length], output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc")
CDO.timstd(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_timstd.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_anomnorm.nc")




In [ ]:
DJF_paths=[]
DJF_anom_paths=[]
DJF_norm_paths=[]
member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_hist, "tas_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_DJF.nc')
    DJF_paths.append(output)

for i in range(member_index_length):
    CDO.timselmean("3", input="-seltimestep,3/467 -selmon,1,2,12 " + input_paths[i], output=DJF_paths[i])
        
CDO.cat(input=DJF_paths[:member_index_length], output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc")
CDO.timstd(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc",output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_timstd.nc", output=mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_anomnorm.nc")




## HIST SLP

In [ ]:

mpige_dir="/Volumes/SPARK/mpi_ge/"
mpige_dir_slp_hist="/Volumes/SPARK/mpi_ge/hist/slp/"
#data_name=os.path.join(data_dir,"test.nc")
#os.chdir(path)

In [ ]:
member_index=[f"{i:03}" for i in range(1,101)]

In [ ]:
# lets see what data we have
arr = os.listdir(mpige_dir_slp_hist)
input_paths=[]
for file in os.listdir(mpige_dir_slp_hist):
    if file.endswith(".nc"):
        print(os.path.join(mpige_dir_slp_hist, file))
        mpige_slp_file=os.path.join(mpige_dir_slp_hist, file)
        input_paths.append(mpige_slp_file)

In [ ]:
jan_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_hist, "psl_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_jan.nc')
    jan_paths.append(output)
    

In [ ]:
jun_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_hist, "psl_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_jun.nc')
    jun_paths.append(output)

In [ ]:
jul_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_hist, "psl_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_jul.nc')
    jul_paths.append(output)

In [ ]:
aug_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_hist, "psl_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_aug.nc')
    aug_paths.append(output)

In [ ]:
feb_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_hist, "psl_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_feb.nc')
    feb_paths.append(output)

In [ ]:
dec_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_hist, "psl_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_dec.nc')
    dec_paths.append(output)


In [ ]:
# calculate dec means and cut them to the region we will use for the dec NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("12", input=" -selyear,1850/2004 " +input_paths[i], output=dec_paths[i])

CDO.cat(input=dec_paths[:member_index_length], output=mpige_dir_slp_hist+"psl_Amon_traintest_dec.nc")

CDO.remapnn("r96x48",input=mpige_dir_slp_hist+"psl_Amon_traintest_dec.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc")
CDO.timstd(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_timstd.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_anomnorm.nc")


In [ ]:
# calculate jan means and cut them to the region we will use for the jan NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("1", input=" -selyear,1851/2005 " +input_paths[i], output=jan_paths[i])

CDO.cat(input=jan_paths[:member_index_length], output=mpige_dir_slp_hist+"psl_Amon_traintest_jan.nc")

CDO.remapnn("r96x48",input=mpige_dir_slp_hist+"psl_Amon_traintest_jan.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc")
CDO.timstd(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc", outputmpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_timstd.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_anomnorm.nc")



In [ ]:
# calculate feb means and cut them to the region we will use for the feb NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("2", input=" -selyear,1851/2005 " +input_paths[i], output=feb_paths[i])

CDO.cat(input=feb_paths[:member_index_length], output=mpige_dir_slp_hist+"psl_Amon_traintest_feb.nc")

CDO.remapnn("r96x48",input=mpige_dir_slp_hist+"psl_Amon_traintest_feb.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc")
CDO.timstd(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_timstd.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_anomnorm.nc")



In [ ]:
# calculate jun means and cut them to the region we will use for the jun NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("6", input=" -selyear,1850/2005 " +input_paths[i], output=jun_paths[i])

CDO.cat(input=jun_paths[:member_index_length], output=mpige_dir_slp_hist+"psl_Amon_traintest_jun.nc")

CDO.remapnn("r96x48",input=mpige_dir_slp_hist+"psl_Amon_traintest_jun.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc")
CDO.timstd(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_timstd.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_anomnorm.nc")




In [ ]:
# calculate jul means and cut them to the region we will use for the jul NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("7", input=" -selyear,1850/2005 " +input_paths[i], output=jul_paths[i])

CDO.cat(input=jul_paths[:member_index_length], output=mpige_dir_slp_hist+"psl_Amon_traintest_jul.nc")

CDO.remapnn("r96x48",input=mpige_dir_slp_hist+"psl_Amon_traintest_jul.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc")
CDO.timstd(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_timstd.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_anomnorm.nc")


In [ ]:
# calculate aug means and cut them to the region we will use for the aug NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("8", input=" -selyear,1850/2005 " +input_paths[i], output=aug_paths[i])

CDO.cat(input=aug_paths[:member_index_length], output=mpige_dir_slp_hist+"psl_Amon_traintest_aug.nc")

CDO.remapnn("r96x48",input=mpige_dir_slp_hist+"psl_Amon_traintest_aug.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc")
CDO.timstd(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_timstd.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_anomnorm.nc")


In [ ]:
JJA_paths=[]
member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_hist, "psl_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_JJA.nc')
    JJA_paths.append(output)for i in range(member_index_length):
    CDO.timselmean("3", input="-selmon,6,7,8 " + input_paths[i], output=JJA_paths[i])
        
CDO.cat(input=JJA_paths[:member_index_length], output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc")
CDO.timstd(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_timstd.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_anomnorm.nc")




In [ ]:
DJF_paths=[]
DJF_anom_paths=[]
DJF_norm_paths=[]
member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_hist, "psl_Amon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_DJF.nc')
    DJF_paths.append(output)

for i in range(member_index_length):
    CDO.timselmean("3", input="-seltimestep,3/467 -selmon,1,2,12 " + input_paths[i], output=DJF_paths[i])
        
CDO.cat(input=DJF_paths[:member_index_length], output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc")
CDO.timstd(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc",output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_timstd.nc")
CDO.sub(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_timstd.nc", output=mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_anomnorm.nc")




## RCP26 T2M 

In [ ]:

mpige_dir="/Volumes/SPARK/mpi_ge/"
mpige_dir_t2m_rcp="/Volumes/SPARK/mpi_ge/rcp26/t2m/"
#data_name=os.path.join(data_dir,"test.nc")
#os.chdir(path)

In [ ]:
member_index=[f"{i:03}" for i in range(1,101)]

In [ ]:
# lets see what data we have
arr = os.listdir(mpige_dir_t2m_rcp)
input_paths=[]
for file in os.listdir(mpige_dir_t2m_rcp):
    if file.endswith(".nc"):
        print(os.path.join(mpige_dir_t2m_rcp, file))
        mpige_t2m_file=os.path.join(mpige_dir_t2m_rcp, file)
        input_paths.append(mpige_t2m_file)

In [ ]:
DJF_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_rcp, "tas_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_DJF.nc')
    DJF_paths.append(output)
    
for i in range(member_index_length):
    CDO.timselmean("3", input="-seltimestep,3/281 -selmon,1,2,12 " + input_paths[i], output=DJF_paths[i])
        
CDO.cat(input=DJF_paths[:member_index_length], output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF.nc",output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow.nc")

CDO.sub(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_timstd.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow_anomnorm.nc")


In [ ]:
JJA_paths=[]
member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_rcp, "tas_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_JJA.nc')
    JJA_paths.append(output)
    
for i in range(member_index_length):
    CDO.timselmean("3", input="-selmon,6,7,8 " + input_paths[i], output=JJA_paths[i])
        
CDO.cat(input=JJA_paths[:member_index_length], output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA.nc",output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA_lowlow.nc")

CDO.sub(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_timstd.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA_lowlow_anomnorm.nc")



In [ ]:
jun_paths=[]
member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_rcp, "tas_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_jun.nc')
    jun_paths.append(output)
    
for i in range(member_index_length):
    CDO.selmon("6", input=input_paths[i], output=jun_paths[i])
        
CDO.cat(input=jun_paths[:member_index_length], output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun.nc",output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun_lowlow.nc")

CDO.sub(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_timstd.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun_lowlow_anomnorm.nc")




In [ ]:
jul_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_rcp, "tas_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_jul.nc')
    jul_paths.append(output)
    
for i in range(member_index_length):
    CDO.selmon("7", input=input_paths[i], output=jul_paths[i])
        
CDO.cat(input=jul_paths[:member_index_length], output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul.nc",output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul_lowlow.nc")

CDO.sub(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_timstd.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul_lowlow_anomnorm.nc")






In [ ]:
aug_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_rcp, "tas_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_aug.nc')
    aug_paths.append(output)
    
for i in range(member_index_length):
    CDO.selmon("8", input=input_paths[i], output=aug_paths[i])
        
CDO.cat(input=aug_paths[:member_index_length], output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug.nc",output=mpige_dir_t2m_rcp"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug_lowlow.nc")


CDO.sub(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_timstd.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug_lowlow_anomnorm.nc")






In [ ]:
dec_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_rcp, "tas_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_dec.nc')
    dec_paths.append(output)
    

In [ ]:
jan_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_rcp, "tas_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_jan.nc')
    jan_paths.append(output)
    

In [ ]:
feb_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_t2m_rcp, "tas_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_feb.nc')
    feb_paths.append(output)
    

In [ ]:
# calculate dec means and cut them to the region we will use for the dec NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("12", input=" -selyear,2006/2098 " +input_paths[i], output=dec_paths[i])

CDO.cat(input=dec_paths[:member_index_length], output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec.nc",output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow.nc")


CDO.sub(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_timstd.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow_anomnorm.nc")


In [ ]:
# calculate DJF means and cut them to the region we will use for the DJF NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("1", input=" -selyear,2007/2099 " +input_paths[i], output=jan_paths[i])

CDO.cat(input=jan_paths[:member_index_length], output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan.nc",output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow.nc")

CDO.sub(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_timstd.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow_anomnorm.nc")



In [ ]:
# calculate DJF means and cut them to the region we will use for the DJF NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("2", input=" -selyear,2007/2099 " +input_paths[i], output=feb_paths[i])

CDO.cat(input=feb_paths[:member_index_length], output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb.nc",output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow.nc")

CDO.sub(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_timstd.nc", output=mpige_dir_t2m_rcp+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow_anomnorm.nc")



## RCP26 SLP

In [ ]:

mpige_dir="/Volumes/SPARK/mpi_ge/"
mpige_dir_t2m_rcp="/Volumes/SPARK/mpi_ge/rcp26/slp/"
#data_name=os.path.join(data_dir,"test.nc")
#os.chdir(path)

In [ ]:
member_index=[f"{i:03}" for i in range(1,101)]

In [ ]:
# lets see what data we have
arr = os.listdir(mpige_dir_slp_rcp)
input_paths=[]
for file in os.listdir(mpige_dir_slp_rcp):
    if file.endswith(".nc"):
        print(os.path.join(mpige_dir_slp_rcp, file))
        mpige_slp_file=os.path.join(mpige_dir_slp_rcp, file)
        input_paths.append(mpige_slp_file)

In [ ]:
DJF_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_rcp, "psl_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_DJF.nc')
    DJF_paths.append(output)
    
for i in range(member_index_length):
    CDO.timselmean("3", input="-seltimestep,3/281 -selmon,1,2,12 " + input_paths[i], output=DJF_paths[i])
        
CDO.cat(input=DJF_paths[:member_index_length], output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF.nc",output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow.nc")

CDO.sub(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_timstd.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow_anomnorm.nc")


In [ ]:
JJA_paths=[]
member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_rcp, "psl_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_JJA.nc')
    JJA_paths.append(output)
    
for i in range(member_index_length):
    CDO.timselmean("3", input="-selmon,6,7,8 " + input_paths[i], output=JJA_paths[i])
        
CDO.cat(input=JJA_paths[:member_index_length], output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA.nc",output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA_lowlow.nc")

CDO.sub(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_timstd.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_JJA_lowlow_anomnorm.nc")



In [ ]:
jun_paths=[]
member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_rcp, "psl_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_jun.nc')
    jun_paths.append(output)
    
for i in range(member_index_length):
    CDO.selmon("6", input=input_paths[i], output=jun_paths[i])
        
CDO.cat(input=jun_paths[:member_index_length], output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun.nc",output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun_lowlow.nc")

CDO.sub(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_timstd.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jun_lowlow_anomnorm.nc")




In [ ]:
jul_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_rcp, "psl_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_jul.nc')
    jul_paths.append(output)
    
for i in range(member_index_length):
    CDO.selmon("7", input=input_paths[i], output=jul_paths[i])
        
CDO.cat(input=jul_paths[:member_index_length], output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul.nc",output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul_lowlow.nc")

CDO.sub(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_timstd.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jul_lowlow_anomnorm.nc")






In [ ]:
aug_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_rcp, "psl_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_aug.nc')
    aug_paths.append(output)
    
for i in range(member_index_length):
    CDO.selmon("8", input=input_paths[i], output=aug_paths[i])
        
CDO.cat(input=aug_paths[:member_index_length], output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug.nc",output=mpige_dir_t2m_rcp"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug_lowlow.nc")


CDO.sub(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_timstd.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_aug_lowlow_anomnorm.nc")






In [ ]:
dec_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_rcp, "psl_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_dec.nc')
    dec_paths.append(output)
    

In [ ]:
jan_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_rcp, "psl_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_jan.nc')
    jan_paths.append(output)
    

In [ ]:
feb_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_slp_rcp, "psl_Amon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_feb.nc')
    feb_paths.append(output)
    

In [ ]:
# calculate dec means and cut them to the region we will use for the dec NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("12", input=" -selyear,2006/2098 " +input_paths[i], output=dec_paths[i])

CDO.cat(input=dec_paths[:member_index_length], output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec.nc",output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow.nc")


CDO.sub(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_timstd.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow_anomnorm.nc")


In [ ]:
# calculate DJF means and cut them to the region we will use for the DJF NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("1", input=" -selyear,2007/2099 " +input_paths[i], output=jan_paths[i])

CDO.cat(input=jan_paths[:member_index_length], output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan.nc",output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow.nc")

CDO.sub(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_timstd.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow_anomnorm.nc")



In [ ]:
# calculate DJF means and cut them to the region we will use for the DJF NAO calculation via EOFs
# 1872/4 = 468, we wanna disregard the first J F and the last D 

for i in range(member_index_length):
    CDO.selmon("2", input=" -selyear,2007/2099 " +input_paths[i], output=feb_paths[i])

CDO.cat(input=feb_paths[:member_index_length], output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb.nc",output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow.nc")

CDO.sub(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_timstd.nc", output=mpige_dir_slp_rcp+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow_anomnorm.nc")



## CONTROL T2M 

In [ ]:

mpige_dir="/Volumes/SPARK/mpi_ge/"
mpige_dir_t2m_ctl="/Volumes/SPARK/mpi_ge/control/t2m/"
#data_name=os.path.join(data_dir,"test.nc")
os.chdir(mpige_dir_t2m_ctl)

In [ ]:
!cdo mergetime tas_Amon* tas_Amon_mpige_control.nc


In [ ]:
!cdo selmon,12 -selyear,1850/3849 tas_Amon_mpige_control.nc tas_Amon_mpige_control_dec.nc
!cdo selmon,1 -selyear,1851/3850 tas_Amon_mpige_control.nc tas_Amon_mpige_control_jan.nc
!cdo selmon,2 -selyear,1851/3850 tas_Amon_mpige_control.nc tas_Amon_mpige_control_feb.nc
!cdo -b 32 timselmean,3 -seltimestep,3/6002 -selmon,1,2,12 tas_Amon_mpige_control.nc tas_Amon_mpige_control_DJF.nc

In [ ]:
!cdo -b 32 selmon,6 tas_Amon_mpige_control.nc t2m/tas_Amon_mpige_control_jun.nc
!cdo -b 32 selmon,7 tas_Amon_mpige_control.nc t2m/tas_Amon_mpige_control_jul.nc
!cdo -b 32 selmon,8 tas_Amon_mpige_control.nc t2m/tas_Amon_mpige_control_aug.nc
!cdo -b 32 yearmean -selmon,6,7,8 tas_Amon_mpige_control.nc tas_Amon_mpige_control_JJA.nc

In [ ]:
CDO.remapnn("r96x48",input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_dec.nc",output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_dec_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jan.nc",output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jan_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_feb.nc",output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_feb_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_DJF.nc",output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_DJF_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jun.nc",output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jun_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jul.nc",output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jul_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_aug.nc",output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_aug_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_JJA.nc",output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_JJA_lowlow.nc")


In [ ]:
CDO.sub(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_dec_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_dec_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_dec_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_timstd.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_dec_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jan_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jan_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jan_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_timstd.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jan_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_feb_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_feb_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_feb_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_timstd.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_feb_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_DJF_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_DJF_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_DJF_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_timstd.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_DJF_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jun_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jun_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jun_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_timstd.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jun_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jul_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jul_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jul_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_timstd.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_jul_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_aug_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_aug_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_aug_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_timstd.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_aug_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_JJA_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_JJA_lowlow_anoms.nc")
CDO.div(input=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_JJA_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_timstd.nc", output=mpige_dir_t2m_ctl+"tas_Amon_mpige_control_JJA_lowlow_anomnorm.nc")


## CONTROL SLP

In [ ]:

mpige_dir="/Volumes/SPARK/mpi_ge/"
mpige_dir_t2m_ctl="/Volumes/SPARK/mpi_ge/control/slp/"
#data_name=os.path.join(data_dir,"test.nc")
os.chdir(mpige_dir_t2m_ctl)

In [ ]:
!cdo mergetime psl_Amon* psl_Amon_mpige_control.nc


In [ ]:
!cdo selmon,12 -selyear,1850/3849 psl_Amon_mpige_control.nc psl_Amon_mpige_control_dec.nc
!cdo selmon,1 -selyear,1851/3850 psl_Amon_mpige_control.nc psl_Amon_mpige_control_jan.nc
!cdo selmon,2 -selyear,1851/3850 psl_Amon_mpige_control.nc psl_Amon_mpige_control_feb.nc
!cdo -b 32 timselmean,3 -seltimestep,3/6002 -selmon,1,2,12 psl_Amon_mpige_control.nc psl_Amon_mpige_control_DJF.nc

In [ ]:
!cdo -b 32 selmon,6 psl_Amon_mpige_control.nc psl_Amon_mpige_control_jun.nc
!cdo -b 32 selmon,7 psl_Amon_mpige_control.nc psl_Amon_mpige_control_jul.nc
!cdo -b 32 selmon,8 psl_Amon_mpige_control.nc psl_Amon_mpige_control_aug.nc
!cdo -b 32 yearmean -selmon,6,7,8 psl_Amon_mpige_control.nc psl_Amon_mpige_control_JJA.nc

In [ ]:
CDO.remapnn("r96x48",input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_dec.nc",output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_dec_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jan.nc",output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jan_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_feb.nc",output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_feb_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_DJF.nc",output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_DJF_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jun.nc",output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jun_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jul.nc",output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jul_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_aug.nc",output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_aug_lowlow.nc")
CDO.remapnn("r96x48",input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_JJA.nc",output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_JJA_lowlow.nc")


In [ ]:
CDO.sub(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_dec_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_dec_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_dec_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_timstd.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_dec_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jan_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jan_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jan_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_timstd.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jan_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_feb_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_feb_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_feb_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_timstd.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_feb_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_DJF_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_DJF_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_DJF_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_timstd.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_DJF_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jun_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jun_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jun_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_timstd.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jun_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jul_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jul_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jul_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_timstd.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_jul_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_aug_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_aug_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_aug_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_timstd.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_aug_lowlow_anomnorm.nc")


In [ ]:
CDO.sub(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_JJA_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_JJA_lowlow_anoms.nc")
CDO.div(input=mpige_dir_slp_ctl+"psl_Amon_mpige_control_JJA_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_timstd.nc", output=mpige_dir_slp_ctl+"psl_Amon_mpige_control_JJA_lowlow_anomnorm.nc")


## HIST THETAO

In [ ]:
# First we have to modulate the available monthly MPI GE netcdf files
# we want to DJF NAO time series for each member
mpige_dir="/Volumes/SPARK/mpi_ge/"
mpige_dir_thetao_hist="/Volumes/SPARK/mpi_ge/hist/thetao/"
#data_name=os.path.join(data_dir,"test.nc")
os.chdir(mpige_dir_thetao_hist)

In [ ]:
!for s in $(seq -w 01 99);do
!echo $s
!cdo sellevel,6 thetao_Omon_MPI-ESM_historical_r0${s}i1850p3_185001-200512.nc thetao7_Omon_MPI-ESM_historical_r0${s}i1850p3_185001-200512.nc
!rm thetao_Omon_MPI-ESM_historical_r0${s}i1850p3_185001-200512.nc
!done
#

In [ ]:
member_index=[f"{i:03}" for i in range(1,101)]

In [ ]:
# lets see what data we have
arr = os.listdir(mpige_dir_thetao)
input_paths=[]
for file in os.listdir(mpige_dir_thetao):
    if file.startswith("thetao7"):
        print(os.path.join(mpige_dir_thetao, file))
        mpige_thetao_file=os.path.join(mpige_dir_thetao, file)
        input_paths.append(mpige_thetao_file)

In [ ]:

nov_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_thetao_hist, "thetao7_OImon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_nov.nc')
    nov_paths.append(output)

    
for i in range(member_index_length):
    CDO.selmon("11", input=" -addc,273.15 -sellevel,6 -selyear,1850/2004 " +input_paths[i], output=nov_paths[i])



CDO.cat(input=nov_paths, output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_low.nc")

CDO.remapbil(mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF.nc",input=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_low.nc",output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_low_atmogrid.nc")

CDO.remapbil("r96x48",input=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_low_atmogrid.nc",output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow.nc")
CDO.setctomiss(0,input=" -mul "+mpige_dir+"supermask_nov.nc" +" "+mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow.nc",output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked.nc"
               
CDO.timstd(input=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked.nc",output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked_timstd.nc")
CDO.sub(input=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked.nc"+" -timmean " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked.nc", output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked_anoms.nc")
CDO.div(input=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked_anoms.nc" + " " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked_timstd.nc", output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked_anomnorm.nc")


In [ ]:
may_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_thetao, "thetao7_OImon_MPI-ESM_historical_r" + str(member_index[i])+'i1850p3_185001-200512_may.nc')
    may_paths.append(output)

for i in range(member_index_length):
    CDO.selmon("5", input=" -addc,273.15 -sellevel,6 -selyear,1850/2005 " +input_paths[i], output=may_paths[i])

CDO.cat(input=may_paths, output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_low.nc")

CDO.remapbil(mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF.nc",input=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_low.nc",output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_low_atmogrid.nc")

CDO.remapbil("r96x48",input=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_low_atmogrid.nc",output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow.nc")

CDO.setctomiss(0,input=" -mul "+mpige_dir+"supermask_may.nc" +" "+mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow.nc",output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked.nc"
               
CDO.timstd(input=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked.nc",output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked_timstd.nc")
CDO.sub(input=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked.nc"+" -timmean " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked.nc", output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked_anoms.nc")
CDO.div(input=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked_anoms.nc" + " " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked_timstd.nc", output=mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked_anomnorm.nc")


## RCP25 THETAO

In [ ]:
# First we have to modulate the available monthly MPI GE netcdf files
# we want to DJF NAO time series for each member
mpige_dir="/Volumes/SPARK/mpi_ge/"
mpige_dir_thetao_rcp="/Volumes/SPARK/mpi_ge/rcp26/thetao/"
#data_name=os.path.join(data_dir,"test.nc")
#os.chdir(mpige_dir_thetao_rcp)

In [ ]:
!for s in $(seq -w 01 99);do
!echo $s
!cdo sellevel,6 thetao_Omon_MPI-ESM_rcp26_r0${s}i2005p3_200601-209912.nc thetao7_Omon_MPI-ESM_rcp26_r0${s}i2005p3_200601-209912.nc
!rm thetao_Omon_MPI-ESM_rcp26_r007i2005p3_200601-209912.nc
!done
#

In [ ]:
member_index=[f"{i:03}" for i in range(1,100)]

In [ ]:
# lets see what data we have
arr = os.listdir(mpige_dir_thetao_rcp)
input_paths=[]
for file in os.listdir(mpige_dir_thetao_rcp):
    if file.startswith("thetao7"):
        print(os.path.join(mpige_dir_thetao_rcp, file))
        mpige_thetao_file=os.path.join(mpige_dir_thetao_rcp, file)
        input_paths.append(mpige_thetao_file)

In [ ]:

nov_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_thetao_rcp, "thetao7_OImon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_nov.nc')
    nov_paths.append(output)


    
for i in range(member_index_length):
    CDO.selmon("11", input=" -addc,273.15 -sellevel,6 -selyear,2006/2098 " +input_paths[i], output=nov_paths[i])



CDO.cat(input=nov_paths, output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_low.nc")

CDO.remapbil(mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF.nc",input=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_low.nc",output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_low_atmogrid.nc")

CDO.remapbil("r96x48",input=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_low_atmogrid.nc",output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_lowlow.nc")
CDO.setctomiss(0,input=" -mul "+mpige_dir+"supermask_nov.nc" +" "+mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_lowlow.nc",output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_lowlow_supermasked.nc")
               
CDO.sub(input=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_lowlow_supermasked.nc"+" -timmean " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked.nc", output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_lowlow_supermasked_anoms.nc")
CDO.div(input=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_lowlow_supermasked_anoms.nc" + " " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked_timstd.nc", output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_nov_lowlow_supermasked_rcp26_anomnorm.nc")


In [ ]:

may_paths=[]

member_index_length = len(member_index)

for i in range(member_index_length):
    output=os.path.join(mpige_dir_thetao_rcp, "thetao7_OImon_MPI-ESM_rcp26_r" + str(member_index[i])+'i2005p3_200601-209912_may.nc')
    may_paths.append(output)


    
for i in range(member_index_length):
    CDO.selmon("5", input=" -addc,273.15 -sellevel,6 -selyear,2006/2098 " +input_paths[i], output=may_paths[i])



CDO.cat(input=may_paths, output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_low.nc")

CDO.remapbil(mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF.nc",input=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_low.nc",output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_low_atmogrid.nc")

CDO.remapbil("r96x48",input=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_low_atmogrid.nc",output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_lowlow.nc")
CDO.setctomiss(0,input=" -mul "+mpige_dir+"supermask_may.nc" +" "+mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_lowlow.nc",output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_lowlow_supermasked.nc")
               
CDO.sub(input=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_lowlow_supermasked.nc"+" -timmean " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked.nc", output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_lowlow_supermasked_anoms.nc")
CDO.div(input=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_lowlow_supermasked_anoms.nc" + " " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked_timstd.nc", output=mpige_dir_thetao_rcp+"thetao7_OImon_traintest_may_lowlow_supermasked_rcp26_anomnorm.nc")


## CONTROL THETAO

In [ ]:
# First we have to modulate the available monthly MPI GE netcdf files
# we want to DJF NAO time series for each member
mpige_dir="/Volumes/SPARK/mpi_ge/"
mpige_dir_thetao_ctl="/Volumes/SPARK/mpi_ge/control/thetao/"
#data_name=os.path.join(data_dir,"test.nc")
os.chdir(mpige_dir_thetao_ctl)

In [ ]:
!cdo mergetime thetao_Omon_MPI-ESM_piControl* thetao_Omon_mpige_control.nc


In [ ]:
!cdo selmon,5 -sellevel,6 -addc,273.15 thetao_Omon_mpige_control.nc thetao_Omon_mpige_control_may.nc
!cdo selmon,11 -sellevel,6 -addc,273.15 -selyear,1850/3849 thetao_Omon_mpige_control.nc thetao_Omon_mpige_control_nov.nc

In [ ]:
CDO.remapbil(mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF.nc",input=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_nov.nc",output=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_nov_atmogrid.nc")

CDO.remapbil("r96x48",input=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_nov_atmogrid.nc",output=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_nov_lowlow.nc")
CDO.setctomiss(0,input=" -mul "+mpige_dir+"supermask_nov.nc" +" "+mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_nov_lowlow.nc",output=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_nov_lowlow_supermasked.nc")
               
CDO.sub(input=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_nov_lowlow_supermasked.nc"+" -timmean " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked.nc", output=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_nov_lowlow_supermasked_anoms.nc")
CDO.div(input=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_nov_lowlow_supermasked_anoms.nc" + " " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked_timstd.nc", output=mpige_dir_thetao_ctl+"thetao_control_nov_lowlow_supermasked_anomnorm.nc")


In [ ]:
CDO.remapbil(mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF.nc",input=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_may.nc",output=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_may_atmogrid.nc")

CDO.remapbil("r96x48",input=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_may_atmogrid.nc",output=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_may_lowlow.nc")
CDO.setctomiss(0,input=" -mul "+mpige_dir+"supermask_may.nc" +" "+mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_may_lowlow.nc",output=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_may_lowlow_supermasked.nc")
               
CDO.sub(input=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_may_lowlow_supermasked.nc"+" -timmean " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked.nc", output=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_may_lowlow_supermasked_anoms.nc")
CDO.div(input=mpige_dir_thetao_ctl+"thetao_Omon_mpige_control_may_lowlow_supermasked_anoms.nc" + " " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_may_lowlow_supermasked_timstd.nc", output=mpige_dir_thetao_ctl+"thetao_control_may_lowlow_supermasked_anomnorm.nc")


# 20CRv3 preprocessing

## 20CR T2M

In [ ]:
# First we have to modulate the available monthly MPI GE netcdf files
# we want to DJF NAO time series for each member
tcr3_dir="/Volumes/SPARK/20crv3/"

In [ ]:
os.chdir(tcr3_dir)

In [ ]:
!cdo -L -b 32 timselmean,3 -seltimestep,3/539 -selmon,1,2,12 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_DJF.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_DJF.nc air.2m.mon.mean_DJF_lowlow.nc
!cdo selyear,1851/1900 air.2m.mon.mean_DJF_lowlow.nc air.2m.mon.mean_DJF_18511900_lowlow.nc


In [ ]:
CDO.sub(input=tcr3_dir+"air.2m.mon.mean_DJF_18511900_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc", output=tcr3_dir+"air.2m.mon.mean_DJF_18511900_lowlow_anoms.nc")
CDO.div(input=tcr3_dir+"air.2m.mon.mean_DJF_18511900_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_timstd.nc", output=tcr3_dir+"air.2m.mon.mean_DJF_18511900_lowlow_anomnorm.nc")


In [ ]:
!cdo -L -b 32 selmon,12 -selyear,1850/1899 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_18501899_dec.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_18501899_dec.nc air.2m.mon.mean_18501899_dec_lowlow.nc


In [ ]:
CDO.sub(input=tcr3_dir+"air.2m.mon.mean_18501899_dec_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow.nc", output=tcr3_dir+"air.2m.mon.mean_dec_18501899_lowlow_anoms.nc")
CDO.div(input=tcr3_dir+"air.2m.mon.mean_dec_18501899_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_timstd.nc", output=tcr3_dir+"air.2m.mon.mean_18501899_dec_lowlow_anomnorm.nc")


In [ ]:
!cdo -L -b 32 selmon,1 -selyear,1851/1900 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_18511900_jan.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_18511900_jan.nc air.2m.mon.mean_18511900_jan_lowlow.nc


In [ ]:
CDO.sub(input=tcr3_dir+"air.2m.mon.mean_18511900_jan_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow.nc", output=tcr3_dir+"air.2m.mon.mean_jan_18511900_lowlow_anoms.nc")
CDO.div(input=tcr3_dir+"air.2m.mon.mean_jan_18511900_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_timstd.nc", output=tcr3_dir+"air.2m.mon.mean_18511900_jan_lowlow_anomnorm.nc")


In [ ]:
!cdo -L -b 32 selmon,2 -selyear,1851/1900 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_18511900_feb.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_18511900_feb.nc air.2m.mon.mean_18511900_feb_lowlow.nc


In [ ]:
CDO.sub(input=tcr3_dir+"air.2m.mon.mean_18511900_feb_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow.nc", output=tcr3_dir+"air.2m.mon.mean_feb_18511900_lowlow_anoms.nc")
CDO.div(input=tcr3_dir+"air.2m.mon.mean_feb_18511900_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_timstd.nc", output=tcr3_dir+"air.2m.mon.mean_18511900_feb_lowlow_anomnorm.nc")


In [ ]:
!cdo -L -b 32 yearmean -selmon,6,7,8  -selyear,1850/1899 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_18501899_JJA.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_18501899_JJA.nc air.2m.mon.mean_18501899_JJA_lowlow.nc


In [ ]:
CDO.sub(input=tcr3_dir+"air.2m.mon.mean_18501899_JJA_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc", output=tcr3_dir+"air.2m.mon.mean_18501899_JJA_lowlow_anoms.nc")
CDO.div(input=tcr3_dir+"air.2m.mon.mean_18501899_JJA_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_timstd.nc", output=tcr3_dir+"air.2m.mon.mean_18501899_JJA_lowlow_anomnorm.nc")


In [ ]:
!cdo -L -b 32 selmon,6 -selyear,1850/1899 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_18501899_jun.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_18501899_jun.nc air.2m.mon.mean_18501899_jun_lowlow.nc

In [ ]:
CDO.sub(input=tcr3_dir+"air.2m.mon.mean_18501899_jun_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow.nc", output=tcr3_dir+"air.2m.mon.mean_18501899_jun_lowlow_anoms.nc")
CDO.div(input=tcr3_dir+"air.2m.mon.mean_18501899_jun_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jun_lowlow_timstd.nc", output=tcr3_dir+"air.2m.mon.mean_18501899_jun_lowlow_anomnorm.nc")


In [ ]:
!cdo -L -b 32 selmon,7 -selyear,1850/1899 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_18501899_jul.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_18501899_jul.nc air.2m.mon.mean_18501899_jul_lowlow.nc

In [ ]:
CDO.sub(input=tcr3_dir+"air.2m.mon.mean_18501899_jul_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow.nc", output=tcr3_dir+"air.2m.mon.mean_18501899_jul_lowlow_anoms.nc")
CDO.div(input=tcr3_dir+"air.2m.mon.mean_18501899_jul_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jul_lowlow_timstd.nc", output=tcr3_dir+"air.2m.mon.mean_18501899_jul_lowlow_anomnorm.nc")


In [ ]:
!cdo -L -b 32 selmon,8 -selyear,1850/1899 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_18501899_aug.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_18501899_aug.nc air.2m.mon.mean_18501899_aug_lowlow.nc

In [ ]:
CDO.sub(input=tcr3_dir+"air.2m.mon.mean_18501899_aug_lowlow.nc"+" -timmean " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow.nc", output=tcr3_dir+"air.2m.mon.mean_18501899_aug_lowlow_anoms.nc")
CDO.div(input=tcr3_dir+"air.2m.mon.mean_18501899_aug_lowlow_anoms.nc" + " " + mpige_dir_t2m_hist+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_aug_lowlow_timstd.nc", output=tcr3_dir+"air.2m.mon.mean_18501899_aug_lowlow_anomnorm.nc")


In [ ]:
## 20th century for evaluation

In [ ]:
!cdo selyear,1901/2015 air.2m.mon.mean_DJF_lowlow.nc air.2m.mon.mean_DJF_19012015_lowlow.nc
!cdo -L -b 32 div -sub air.2m.mon.mean_DJF_19012015_lowlow.nc -timmean air.2m.mon.mean_DJF_19012015_lowlow.nc -timstd air.2m.mon.mean_DJF_19012015_lowlow.nc air.2m.mon.mean_DJF_19012015_lowlow_anomnorm.nc

In [ ]:
!cdo -L -b 32 selmon,12 -selyear,1900/2014 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_19002014_dec.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_19002014_dec.nc air.2m.mon.mean_19002014_dec_lowlow.nc
!cdo -L -b 32 div -sub air.2m.mon.mean_19002014_dec_lowlow.nc -timmean air.2m.mon.mean_19002014_dec_lowlow.nc -timstd air.2m.mon.mean_19002014_dec_lowlow.nc air.2m.mon.mean_19002014_dec_lowlow_anomnorm.nc

In [ ]:
!cdo -L -b 32 selmon,1 -selyear,1901/2015 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_19012015_jan.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_19012015_jan.nc air.2m.mon.mean_19012015_jan_lowlow.nc
!cdo -L -b 32 div -sub air.2m.mon.mean_19012015_jan_lowlow.nc -timmean air.2m.mon.mean_19012015_jan_lowlow.nc -timstd air.2m.mon.mean_19012015_jan_lowlow.nc air.2m.mon.mean_19012015_jan_lowlow_anomnorm.nc

In [ ]:
!cdo -L -b 32 selmon,2 -selyear,1901/2015 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_19012015_feb.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_19012015_feb.nc air.2m.mon.mean_19012015_feb_lowlow.nc
!cdo -L -b 32 div -sub air.2m.mon.mean_19012015_feb_lowlow.nc -timmean air.2m.mon.mean_19012015_feb_lowlow.nc -timstd air.2m.mon.mean_19012015_feb_lowlow.nc air.2m.mon.mean_19012015_feb_lowlow_anomnorm.nc

In [ ]:
!cdo -L -b 32 yearmean -selmon,6,7,8  -selyear,1900/2015 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_19002015_JJA.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_19002015_JJA.nc air.2m.mon.mean_19002015_JJA_lowlow.nc
!cdo -L -b 32 div -sub air.2m.mon.mean_19002015_JJA_lowlow.nc -timmean air.2m.mon.mean_19002015_JJA_lowlow.nc -timstd air.2m.mon.mean_19002015_JJA_lowlow.nc air.2m.mon.mean_19002015_JJA_lowlow_anomnorm.nc

In [ ]:
!cdo -L -b 32 selmon,6  -selyear,1900/2015 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_19002015_jun.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_19002015_jun.nc air.2m.mon.mean_19002015_jun_lowlow.nc
!cdo -L -b 32 div -sub air.2m.mon.mean_19002015_jun_lowlow.nc -timmean air.2m.mon.mean_19002015_jun_lowlow.nc -timstd air.2m.mon.mean_19002015_jun_lowlow.nc air.2m.mon.mean_19002015_jun_lowlow_anomnorm.nc

In [ ]:
!cdo -L -b 32 selmon,7  -selyear,1900/2015 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_19002015_jul.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_19002015_jul.nc air.2m.mon.mean_19002015_jul_lowlow.nc
!cdo -L -b 32 div -sub air.2m.mon.mean_19002015_jul_lowlow.nc -timmean air.2m.mon.mean_19002015_jul_lowlow.nc -timstd air.2m.mon.mean_19002015_jul_lowlow.nc air.2m.mon.mean_19002015_jul_lowlow_anomnorm.nc

In [ ]:
!cdo -L -b 32 selmon,8 -selyear,1900/2015 -selname,air air.2m.mon.mean.nc air.2m.mon.mean_19002015_aug.nc
!cdo -b 32 remapbil,r96x48 air.2m.mon.mean_19002015_aug.nc air.2m.mon.mean_19002015_aug_lowlow.nc
!cdo -L -b 32 div -sub air.2m.mon.mean_19002015_aug_lowlow.nc -timmean air.2m.mon.mean_19002015_aug_lowlow.nc -timstd air.2m.mon.mean_19002015_aug_lowlow.nc air.2m.mon.mean_19002015_aug_lowlow_anomnorm.nc

## 20CR SLP

In [ ]:
!cdo -L -b 32 timselmean,3 -seltimestep,3/539 -selmon,1,2,12 -selname,prmsl prmsl.mon.mean.nc prmsl.mon.mean_DJF.nc
!cdo -b 32 remapbil,r96x48 prmsl.mon.mean_DJF.nc prmsl.mon.mean_DJF_lowlow.nc
!cdo selyear,1851/1900 prmsl.mon.mean_DJF_lowlow.nc prmsl.mon.mean_DJF_18511900_lowlow.nc


In [ ]:
CDO.sub(input=tcr3_dir+"prmsl.mon.mean_DJF_18511900_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow.nc", output=tcr3_dir+"prmsl.mon.mean_DJF_18511900_lowlow_anoms.nc")
CDO.div(input=tcr3_dir+"prmsl.mon.mean_DJF_18511900_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_timstd.nc", output=tcr3_dir+"prmsl.mon.mean_DJF_18511900_lowlow_anomnorm.nc")


In [ ]:
!cdo -L -b 32 yearmean -selmon,6,7,8  -selyear,1850/1899 -selname,prmsl prmsl.mon.mean.nc prmsl.mon.mean_18501899_JJA.nc
!cdo -b 32 remapbil,r96x48 prmsl.mon.mean_18501899_JJA.nc prmsl.mon.mean_18501899_JJA_lowlow.nc


In [ ]:
CDO.sub(input=tcr3_dir+"prmsl.mon.mean_18501899_JJA_lowlow.nc"+" -timmean " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow.nc", output=tcr3_dir+"prmsl.mon.mean_18501899_JJA_lowlow_anoms.nc")
CDO.div(input=tcr3_dir+"prmsl.mon.mean_18501899_JJA_lowlow_anoms.nc" + " " + mpige_dir_slp_hist+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_JJA_lowlow_timstd.nc", output=tcr3_dir+"prmsl.mon.mean_18501899_JJA_lowlow_anomnorm.nc")


In [ ]:
## 20th century for evaluation

In [ ]:
!cdo selyear,1901/2015 prmsl.mon.mean_DJF_lowlow.nc prmsl.mon.mean_DJF_19012015_lowlow.nc
!cdo -L -b 32 div -sub prmsl.mon.mean_DJF_19012015_lowlow.nc -timmean prmsl.mon.mean_DJF_19012015_lowlow.nc -timstd prmsl.mon.mean_DJF_19012015_lowlow.nc prmsl.mon.mean_DJF_19012015_lowlow_anomnorm.nc

In [ ]:
!cdo -L -b 32 yearmean -selmon,6,7,8  -selyear,1900/2015 -selname,prmsl prmsl.mon.mean.nc prmsl.mon.mean_19002015_JJA.nc
!cdo -b 32 remapbil,r96x48 prmsl.mon.mean_19002015_JJA.nc prmsl.mon.mean_19002015_JJA_lowlow.nc
!cdo -L -b 32 div -sub prmsl.mon.mean_19002015_JJA_lowlow.nc -timmean prmsl.mon.mean_19002015_JJA_lowlow.nc -timstd prmsl.mon.mean_19002015_JJA_lowlow.nc prmsl.mon.mean_19002015_JJA_lowlow_anomnorm.nc

## CODA SST

In [ ]:
!cdo -L -b 32 selmon,5 -addc,273.15 -selyear,1850/2015 -selname,sst sst.mon.mean.nc sst.mon.mean_1850_may.nc
!cdo remapbil,r96x48c sst.mon.mean_1850_may.nc sst.mon.mean_1850_may_lowlow.nc
!cdo selyear,1850/1899 sst.mon.mean_1850_may_lowlow.nc sst.mon.mean_18501899_may_lowlow.nc
!cdo -L -b 32 setctomiss,0 -mul supermask_may.nc sst.mon.mean_18501899_may_lowlow.nc sst.mon.mean_18501899_may_lowlow_supermasked.nc

In [ ]:
CDO.sub(input=tcr3_dir+"sst.mon.mean_18501899_may_lowlow_supermasked.nc"+" -timmean " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked.nc", output=tcr3_dir+"sst.mon.mean_18501899_may_lowlow_supermasked_anoms.nc")
CDO.div(input=tcr3_dir+"sst.mon.mean_18501899_may_lowlow_supermasked_anoms.nc" + " " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked_timstd.nc", output=tcr3_dir+"sst.mon.mean_18501899_may_lowlow_supermasked_anomnorm.nc")


In [ ]:
!cdo -L -b 32 selmon,11 -addc,273.15 -selyear,1850/2015 -selname,sst sst.mon.mean.nc sst.mon.mean_1850_nov.nc
!cdo remapbil,r96x48c sst.mon.mean_1850_nov.nc sst.mon.mean_1850_nov_lowlow.nc
!cdo selyear,1850/1899 sst.mon.mean_1850_may_lowlow.nc sst.mon.mean_18501899_nov_lowlow.nc
!cdo -L -b 32 setctomiss,0 -mul supermask_nov.nc sst.mon.mean_18501899_nov_lowlow.nc sst.mon.mean_18501899_nov_lowlow_supermasked.nc

In [ ]:
CDO.sub(input=tcr3_dir+"sst.mon.mean_18501899_nov_lowlow_supermasked.nc"+" -timmean " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked.nc", output=tcr3_dir+"sst.mon.mean_18501899_nov_lowlow_supermasked_anoms.nc")
CDO.div(input=tcr3_dir+"sst.mon.mean_18501899_nov_lowlow_supermasked_anoms.nc" + " " + mpige_dir_thetao_hist+"thetao7_OImon_traintest_nov_lowlow_supermasked_timstd.nc", output=tcr3_dir+"sst.mon.mean_18501899_nov_lowlow_supermasked_anomnorm.nc")


In [ ]:
## 20th century for evaluation

In [ ]:
!cdo selyear,1900/2015 sst.mon.mean_1850_may_lowlow.nc sst.mon.mean_19002015_may_lowlow.nc
!cdo -L -b 32 setctomiss,0 -mul supermask_may.nc sst.mon.mean_19002015_may_lowlow.nc sst.mon.mean_19002015_may_lowlow_supermasked.nc
!cdo -L -b 32 div -sub sst.mon.mean_19002015_may_lowlow_supermasked.nc -timmean sst.mon.mean_19002015_may_lowlow_supermasked.nc -timstd sst.mon.mean_19002015_may_lowlow_supermasked.nc sst.mon.mean_19002015_may_lowlow_supermasked_anomnorm.nc

In [ ]:

!cdo selyear,1900/2014 sst.mon.mean_1850_may_lowlow.nc sst.mon.mean_19002014_nov_lowlow.nc
!cdo -L -b 32 setctomiss,0 -mul supermask_nov.nc sst.mon.mean_19002014_nov_lowlow.nc sst.mon.mean_19002014_nov_lowlow_supermasked.nc
!cdo -L -b 32 div -sub sst.mon.mean_19002014_nov_lowlow_supermasked.nc -timmean sst.mon.mean_19002014_nov_lowlow_supermasked.nc -timstd sst.mon.mean_19002014_nov_lowlow_supermasked.nc sst.mon.mean_19002014_nov_lowlow_supermasked_anomnorm.nc